In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import os
import importlib
import logging
import keras.backend as K
from functions import load_train_data

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=20000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
    print(e)

2 Physical GPUs, 1 Logical GPU


In [2]:
#3flavor_clean, 3flavor_poisson, nsi_clean, nsi_poisson
learn_target = 'nsi_poisson'

x_train, y_train, x_val, y_val = load_train_data(learn_target)

In [3]:
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = tf.keras.Sequential([
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)/10))])
    return prior_model

def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = tf.keras.Sequential([
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ), tfp.layers.MultivariateNormalTriL(n),])
    return posterior_model

In [4]:
def create_bnn_model(train_size):
    inputs = tf.keras.Input(shape=(len(x_train[0]),), name = 'input')
    features = tf.keras.layers.BatchNormalization()(inputs)
    features = tf.keras.layers.Dense(256, activation='relu')(features)
    features = tf.keras.layers.Dense(128, activation='relu')(features)

    for units in [32]:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)
    features = tf.keras.layers.Dense(16, activation='relu')(features)
    features = tf.keras.layers.Dense(len(y_train[0]), activation='linear')(features)
    outputs = features
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

alpha = np.array([1, 20, 1, 1]) if '3flavor' in learn_target else np.array([1, 20, 1, 1, 1, 1, 1])
alpha = alpha/sum(alpha)

def custom_mse(y_true, y_pred):
    loss = K.square(y_pred - y_true)  
    loss = loss * alpha      
    loss = K.sum(loss, axis=1)        
    return loss

In [5]:
bnn = create_bnn_model(len(x_train))
bnn.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
    loss=custom_mse,
    metrics=[tf.keras.metrics.MeanSquaredError()],
)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


In [6]:
bnn.fit(x_train, y_train,
            validation_data=(x_val, y_val),
            batch_size=1024,
            epochs=100,
            verbose=1,
            shuffle = True,
            )

Epoch 1/100
8790/8790 [==============================] - 72s 8ms/step - loss: 0.1012 - mean_squared_error: 0.2361 - val_loss: 0.0824 - val_mean_squared_error: 0.1788
Epoch 2/100
8790/8790 [==============================] - 70s 8ms/step - loss: 0.0702 - mean_squared_error: 0.1322 - val_loss: 0.0621 - val_mean_squared_error: 0.1005
Epoch 3/100
8790/8790 [==============================] - 68s 8ms/step - loss: 0.0597 - mean_squared_error: 0.0924 - val_loss: 0.0585 - val_mean_squared_error: 0.0877
Epoch 4/100
8790/8790 [==============================] - 69s 8ms/step - loss: 0.0562 - mean_squared_error: 0.0803 - val_loss: 0.0555 - val_mean_squared_error: 0.0771
Epoch 5/100
8790/8790 [==============================] - 69s 8ms/step - loss: 0.0542 - mean_squared_error: 0.0734 - val_loss: 0.0552 - val_mean_squared_error: 0.0772
Epoch 6/100
8790/8790 [==============================] - 68s 8ms/step - loss: 0.0531 - mean_squared_error: 0.0693 - val_loss: 0.0526 - val_mean_squared_error: 0.0676
Epoc

In [7]:
bnn.save_weights('./bnn/' + learn_target + '/weight_1.h5')